In [1]:
import pandas as pd
from konlpy.tag import Kkma
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [2]:
def rmEmoji(data):
    return data.encode('euc-kr', 'ignore').decode('euc-kr')

# 한옥 데이터 전처리

In [3]:
hanok = pd.read_csv('CSV/거제도펜션.csv')

In [4]:
sukso_recommendation = pd.read_csv('CSV/숙소추천.csv')
sukso = pd.read_csv('CSV/숙소.csv')

In [5]:
# hanok = hanok.drop('Unnamed: 0', axis = 1)

# hanok_hash 전처리

In [6]:
hanok_hash_list = [] # 비어있는 리스트 만들기 
for txt in hanok['hashtag'] : 
    del_br = txt.strip("[]")
    del_br = del_br.replace("#","")
    del_br = del_br.replace(",","")
    del_br = del_br.replace("'","")
    
    hanok_hash_list.append(del_br) #값 채우기

In [7]:
hanok_hash_series = pd.Series(hanok_hash_list)

In [8]:
hanok_tf = hanok_hash_series.str.contains("숙소") == 0
hanok_tf = hanok_hash_series.str.contains("숙소추천") == 0
hanok_hash_series = hanok_hash_series[hanok_tf]

In [9]:
hanok_hash_series.reset_index(drop = True, inplace = True)

# hanok_df 전처리

In [10]:
hanok_df = hanok[hanok['hashtag'].str.contains("숙소") ==0]
hanok_df = hanok[hanok['hashtag'].str.contains("숙소추천") ==0]

In [11]:
index_del = hanok_df[hanok_df['hashtag'] == '[]'].index
hanok_df = hanok_df.drop(index_del)

In [12]:
hanok_df.reset_index(drop = True, inplace = True)

In [13]:
hanok_df['hashtag'] = hanok_hash_series

In [14]:
for i in range(len(hanok_df['hashtag'])):
    if(len(hanok_df['hashtag'].iloc[i])<=2):
        hanok_df['hashtag'].iloc[i] = rmEmoji(hanok_df['hashtag'].iloc[i])

C:\Users\morni\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [15]:
index_del = hanok_df[hanok_df['hashtag'] == ''].index
hanok_df = hanok_df.drop(index_del)

In [16]:
hanok_df.reset_index(drop = True, inplace = True)

In [17]:
hanok_df

,id,date,like,text,hashtag,src,target
0,gamsung_curation,2021-05-26 00:00:00+00:00,"1,174","✅ . . 에메랄드빛 거제 바다를 바라보며 힐링하기 좋은 ""바람의언덕"" 야자수로 둘...",거제_바람의언덕리조트 감성숙소_거제 감성숙소_경상남도 경상남도 거제 바람의언덕리조트...,https://scontent-ssn1-1.cdninstagram.com/v/t51...,1
1,windyhill_geoje,2021-06-17 00:00:00+00:00,"2,459",여기가 한국 맞긴 한가요😳 국내에도 이렇게 예쁜 숙소가 있다구요! 가만히 객실에만 ...,바람의언덕리조트 썬트리팜 거제도펜션 거제펜션 거제수국 거제히노끼펜션 경남여행 거제도...,https://scontent-ssn1-1.cdninstagram.com/v/t51...,1
2,yumhee_,2021-05-25 00:00:00+00:00,650,/ 오랜만에 가족과 힐링타임💗 바로 앞엔 ...,얌희_거제 얌희_경남 거제도여행 거제도펜ᄉ...,NaN,1
3,violin_ara,2021-06-16 00:00:00+00:00,847,멀리까지 가는데 1박만 하기는 아쉬워서 급하게 서치해서 예약한 곳인데 이곳이 이번 ...,뚜호으노네집_여행,NaN,1
4,sua_play,2021-05-28 00:00:00+00:00,"1,544",: ⠀ 어제 스토리 올리고 문의 폭주한 숙소는 거제도 바람의언덕 리조트에요!😍 뷰에...,수아네여행,NaN,1
...,...,...,...,...,...,...,...
4984,geojenoingwabada,2021-03-26 00:00:00+00:00,26,2021.3.26 ㆍ ㆍ 열기와 볼락입니다 🐟🎣⛵ ㆍ ㆍ 14주,거제도여행 가족여행 거제도펜션 엄빠따라다닌다고 찬율이고생했쒀,NaN,1
4985,s0ra_s,2021-03-26 00:00:00+00:00,276,벚꽃 좋아 💕 14주,호텔상상 거제도가볼만한곳 거제도숙소 거제도호텔 거제호텔 거제도펜션 거제펜션 거제도풀...,NaN,1
4986,adstation_master,2021-03-26 00:00:00+00:00,148,"가을, 겨울에도 야외 수영장 온수풀 개장한다는데, 이번 가을에도 재방문 해야겠네요 ...",거제도펜션 거제펜션 거제풀빌라 거제도풀빌라 거제여행 거제도여행 거제가볼만한곳 거제도...,NaN,1
4987,geojeislandpension,2021-03-26 00:00:00+00:00,7,@mj_log 29번째 생일 축하해주신분들 감사합니다🤍 덕분에 찐한 생일 보냈습니다...,소니💌거제 바람의언덕리조트 사유지,NaN,1


In [18]:
hanok_df_vc = hanok_df['id'].value_counts().to_frame()

In [19]:
hanok_df_vc_over13 = hanok_df_vc[hanok_df_vc['id']>12].index

In [20]:
hanok_over13_name = hanok_df_vc_over13.unique()

In [21]:
for i in range(len(hanok_over13_name)):
    hanok_df = hanok_df.loc[hanok_df["id"].str.contains(hanok_over13_name[i]) == 0]

In [22]:
hanok_df_vc = hanok_df['id'].value_counts().to_frame()

In [23]:
hanok_df_vc[hanok_df_vc['id']>12].index

Index([], dtype='object')

In [24]:
hanok_df.reset_index(drop = True, inplace = True)

# sukso + sukso_reco : MERGE

In [25]:
# sukso_recommendation = pd.read_csv('숙소추천.csv')
# sukso = pd.read_csv('숙소.csv')

In [26]:
sukso = sukso[sukso['hashtag'].str.contains('숙소추천')==0]

In [27]:
sukso_merge = pd.concat([sukso_recommendation,sukso] , axis= 0)

In [28]:
index_del2 = sukso_merge[sukso_merge['hashtag'] == '[]'].index
sukso_merge = sukso_merge.drop(index_del2)

In [29]:
sukso_merge_list = [] # 비어있는 리스트 만들기 
for txt in sukso_merge['hashtag'] : 
    del_br = txt.strip("[]")
    del_br = del_br.replace("#"," ")
    del_br = del_br.replace(",","")
    del_br = del_br.replace("'","")
    sukso_merge_list.append(del_br) #값 채우기

In [30]:
sukso_merge_series = pd.Series(sukso_merge_list)

In [31]:
sukso_merge_series= sukso_merge_series.str.contains("거제도펜션") ==0
sukso_merge_series= sukso_merge_series[sukso_merge_series] 

In [32]:
sukso_merge.reset_index(drop = True, inplace = True)

In [33]:
sukso_merge['hashtag'] =sukso_merge_list

In [34]:
for i in range(len(sukso_merge['hashtag'])):
    if(len(sukso_merge['hashtag'].iloc[i])<=2):
        sukso_merge['hashtag'].iloc[i] = rmEmoji(sukso_merge['hashtag'].iloc[i])

C:\Users\morni\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [35]:
index_del = sukso_merge[sukso_merge['hashtag'] == ''].index
sukso_merge = sukso_merge.drop(index_del)

In [36]:
sukso_merge.reset_index(drop = True, inplace = True)

In [37]:
sukso_merge

,id,date,like,text,hashtag,src,target
0,busansomang,2021-06-21 00:00:00+00:00,"2,086","감성숙소 모음리스트✈️ 갬성, 힐링 모든 면에서 대박인 제쥬도 존예 숙소 다 모아와...",제주 제주 제주도 숙소 제주숙소 숙소추천 제주숙소추천 제주여행 제주...,https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
1,zzazang_,2021-06-26 00:00:00+00:00,"3,385",이거 하나보고 예약했던 숙소 🪨🪵🐷🥩🥬🍹💖 . . . 2일,제주여행 제주도여행 제주도 제주 제주숙소 제주숙소추천 제주독채 제주독...,https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
2,ejiyang,2021-06-09 00:00:00+00:00,"1,785",문의주신 숙소 공유해드려요♥️ 서귀포에 위치한 이에요🌼 이번 제주는 서귀포쪽에 머물...,부영호텔,https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
3,so__hyeon,2021-06-26 00:00:00+00:00,"14,145",여기 진짜 미쳤다. 너무너무 좋다는 말 밖에🏕🍃 우리 사이트 바로 앞으로 바람 솔솔...,캠핑 캠핑장추천 캠핑장 캠핑스타...,https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
4,stayfolio\n인증됨,2021-06-24 00:00:00+00:00,"1,892","인증됨 작고(釐毫) 소박한 즐거움(小樂)을 더하다, 화려하게 꾸며진 공간은 많다. ...",이호소락 스테이폴리오 스테이폴리오_서울 북촌 노천탕,https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
...,...,...,...,...,...,...,...
7122,mhc022012,2021-03-30 00:00:00+00:00,영상,Happy pottery studio Jeju 행복공방 에치비 도자기체험 제주 Ma...,체험 가볼만한곳 제주 서귀포 도자기 도예 귤 감귤 중문 숙소 카...,NaN,0
7123,seo_yeong_star,2021-03-30 00:00:00+00:00,246,방문자리뷰에 올려주신 후기 사진이예요 파란하늘과 마루방을 너무 이쁘게 담아주셔서 가...,누마루 파란하늘 방문자리뷰 스테이황룡 한옥스타그램 한옥스테이 한옥게스트...,https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
7124,___ju1004,2021-03-30 00:00:00+00:00,24,제주도에서의 하루^^ 제주도도 황사가 장난아님...;;; . . . 13주,정방폭포 제주도 황사 신행 신혼여행 숙소,https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
7125,hyejini_vv,2021-03-30 00:00:00+00:00,20,제주 여행 여덟번째기록📝 노잼여행을 유잼여행으로 만들어주신 @youngmi__s2 ...,마지막 기록 제주 여행 뚜벅이 혼여행 마지막피드 즐거웠다 날씨는 ...,https://scontent-ssn1-1.cdninstagram.com/v/t51...,0


# 머신러닝 학습

In [38]:
sukso_merge_data= sukso_merge.iloc[:, [4,6]]
hanok_data = hanok_df.iloc[:, [4,6]]

In [39]:
hanok_plus_sukso_merge = pd.concat([hanok_data,sukso_merge_data], axis = 0)

In [40]:
hanok_plus_sukso_merge_X = hanok_plus_sukso_merge.iloc[:, 0]
hanok_plus_sukso_merge_y = hanok_plus_sukso_merge.iloc[:, 1]

In [41]:
from sklearn.model_selection import train_test_split

In [42]:
X_train, X_test, y_train, y_test = train_test_split(hanok_plus_sukso_merge_X, # 문제데이터
                                                    hanok_plus_sukso_merge_y,  #정답데이터
                                                   random_state = 3,  # 섞는 방법
                                                   test_size =0.3)

In [43]:
from sklearn.feature_extraction.text import CountVectorizer
testCV = CountVectorizer()

In [44]:
testCV.fit(X_train, y_train)

CountVectorizer()

In [45]:
X_train_CV = testCV.transform(X_train)
X_test_CV = testCV.transform(X_test)

In [46]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
svm = LinearSVC()

In [47]:
svm_result = cross_val_score(svm, X_train_CV, y_train, cv=5)
svm_result.mean()

0.9837934362934362

In [48]:
svm.fit(X_train_CV,y_train)

LinearSVC()

# 숙소_숙소추천에서 특정 카테고리 분류하기

In [49]:
sukso_recommendation = pd.read_csv('숙소추천.csv')
sukso = pd.read_csv('숙소.csv')

In [50]:
sukso = sukso[sukso['hashtag'].str.contains('숙소추천')==0]

In [51]:
sukso_merge = pd.concat([sukso_recommendation,sukso] , axis= 0)

In [52]:
index_del2 = sukso_merge[sukso_merge['hashtag'] == '[]'].index
sukso_merge = sukso_merge.drop(index_del2)

In [53]:
sukso_merge_list = [] # 비어있는 리스트 만들기 
for txt in sukso_merge['hashtag'] : 
    del_br = txt.strip("[]")
    del_br = del_br.replace("#"," ")
    del_br = del_br.replace(",","")
    del_br = del_br.replace("'","")
    sukso_merge_list.append(del_br) #값 채우기

In [54]:
sukso_merge_series = pd.Series(sukso_merge_list)

In [55]:
sukso_merge.reset_index(drop = True, inplace = True)

In [56]:
sukso_merge['hashtag'] =sukso_merge_list

In [57]:
X_train_CV = testCV.transform(sukso_merge.iloc[:, 4])

In [58]:
predict_array = svm.predict(X_train_CV)

In [59]:
one_numpy = np.where(predict_array==1)

In [60]:
one_numpy

(array([ 243,  844, 1264, 1488, 2242, 2706, 3367, 3609, 4948], dtype=int64),)

In [61]:
one_list = list(one_numpy)

In [62]:
one_series = pd.Series(one_list)

In [63]:
one_list = one_list[0].tolist()

In [64]:
predict_df = sukso_merge.iloc[(one_list)]

In [65]:
index_del = predict_df[predict_df['like'] == '영상'].index
predict_df = predict_df.drop(index_del)

In [66]:
predict_df.keys()

Index(['id', 'date', 'like', 'text', 'hashtag', 'src', 'target'], dtype='object')

In [67]:
predict_df.reset_index(drop = True, inplace = True)

In [68]:
for i in range(len(predict_df)):
    predict_df['like'].iloc[i] = predict_df['like'].iloc[i].replace(",","")
    predict_df['like'].iloc[i] = int(predict_df['like'].iloc[i])

C:\Users\morni\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [69]:
predict_df = predict_df.sort_values(by = ['like'], axis = 0, ascending = False)

In [70]:
predict_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9 entries, 1 to 6
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       9 non-null      object
 1   date     9 non-null      object
 2   like     9 non-null      object
 3   text     9 non-null      object
 4   hashtag  9 non-null      object
 5   src      9 non-null      object
 6   target   9 non-null      int64 
dtypes: int64(1), object(6)
memory usage: 576.0+ bytes


In [71]:
predict_df.to_csv("거제도펜션_정답목록.csv", encoding="utf-8-sig")

In [72]:
print("학습시킨 데이터의 양 : ", len(hanok))

학습시킨 데이터의 양 :  5886
